# Numerical linear algebra Project
Implements numerical methods, specifically LU decomposition and Gaussian elimination with pivoting, to solve systems of linear equations using Python and NumPy.

In [19]:
import numpy as np

## Part 1: LU method to solve the system AX=b where A is an n*n matrix

The code below checks if LU decomposition is possible for a square matrix A. It iterates through each leading principal minor of A and calculates its determinant. If any determinant is close to zero, it returns False; otherwise, it returns True. The function ensures that all leading principal minors are non-zero for LU decomposition to be feasible.

In [20]:
def is_lu_possible(A):
    n = len(A)
    for i in range(n):
        minor = np.linalg.det(A[:i+1, :i+1])
        if np.isclose(minor, 0.0):
            return False
    return True

The lu_decomposition function checks if LU decomposition is possible for a square matrix A. If possible, it initializes lower triangular matrix L as the identity matrix and upper triangular matrix U as a zero matrix. It then iteratively calculates the elements of L and U using nested loops, printing the progress at each step. The function returns the decomposed matrices L and U. If LU decomposition is not possible, it prints a message and returns None, None.

In [21]:
def lu_decomposition(A):
    if not is_lu_possible(A):
        print("LU decomposition is not possible due to zero leading principal minors.")
        return None, None
    
    n = len(A)
    L = np.eye(n)
    U = np.zeros((n, n))

    for i in range(n):
        for j in range(i, n):
            U[i, j] = A[i, j] - sum(L[i, k] * U[k, j] for k in range(i))
            print(f"Step {i + 1} (U): u({i+1}{j+1}) = {U[i, j]}")
        for j in range(i + 1, n):
            L[j, i] = (A[j, i] - sum(L[j, k] * U[k, i] for k in range(i))) / U[i, i]
            print(f"Step {i + 1} (L): l({j+1}{i+1}) = {L[j, i]}")

        print(f"Step {i + 1}:\nL:\n{L}\nU:\n{U}\n")

    return L, U

The solve_lu function solves a system of linear equations given the LU decomposition of the coefficient matrix. It performs forward substitution for the lower triangular system Ly = b and back substitution for the upper triangular system Ux = y. The intermediate and final solutions are printed. The function returns the solution vector x.

In [22]:
def solve_lu(L, U, b):
    n = len(b)
    # Solve Ly = b for y
    y = np.zeros(n)
    for i in range(n):
        y[i] = b[i] - sum(L[i, k] * y[k] for k in range(i))
    
    print(f"Solved Ly = b, y = {y}\n")

    # Solve Ux = y for x
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - sum(U[i, k] * x[k] for k in range(i + 1, n))) / U[i, i]

    print(f"Solved Ux = y, x = {x}\n")

    return x

Example usage 1

In [23]:
A = np.array([[1, 2, 3],
              [2, 4, 8],
              [-1, 0, 1]])

b = np.array([1, 0, -1])

if is_lu_possible(A):
    L, U = lu_decomposition(A)
    x = solve_lu(L, U, b)
    print("Final Solution x:", x)
else:
    print("LU decomposition is not possible due to a zero leading principal minor.")

LU decomposition is not possible due to a zero leading principal minor.


Example usage 2

In [24]:
A = np.array([[2, -1, 0],
              [-1, 2, -1],
              [0, -1, 2]])

b = np.array([1, 0, -1])

if is_lu_possible(A):
    L, U = lu_decomposition(A)
    x = solve_lu(L, U, b)
    print("Final Solution x:", x)
else:
    print("LU decomposition is not possible due to a zero leading principal minor.")

Step 1 (U): u(11) = 2.0
Step 1 (U): u(12) = -1.0
Step 1 (U): u(13) = 0.0
Step 1 (L): l(21) = -0.5
Step 1 (L): l(31) = 0.0
Step 1:
L:
[[ 1.   0.   0. ]
 [-0.5  1.   0. ]
 [ 0.   0.   1. ]]
U:
[[ 2. -1.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]

Step 2 (U): u(22) = 1.5
Step 2 (U): u(23) = -1.0
Step 2 (L): l(32) = -0.6666666666666666
Step 2:
L:
[[ 1.          0.          0.        ]
 [-0.5         1.          0.        ]
 [ 0.         -0.66666667  1.        ]]
U:
[[ 2.  -1.   0. ]
 [ 0.   1.5 -1. ]
 [ 0.   0.   0. ]]

Step 3 (U): u(33) = 1.3333333333333335
Step 3:
L:
[[ 1.          0.          0.        ]
 [-0.5         1.          0.        ]
 [ 0.         -0.66666667  1.        ]]
U:
[[ 2.         -1.          0.        ]
 [ 0.          1.5        -1.        ]
 [ 0.          0.          1.33333333]]

Solved Ly = b, y = [ 1.          0.5        -0.66666667]

Solved Ux = y, x = [ 0.5  0.  -0.5]

Final Solution x: [ 0.5  0.  -0.5]


## Part 2: Gaussian elimination with pivoting method to solve the system AX=b


The code below defines a function gaussian_elimination_pivoting to solve a system of linear equations Ax=b using Gaussian elimination with partial pivoting. It converts input matrices to float64, creates an augmented matrix, and performs forward elimination with partial pivoting. The intermediate steps, including pivot rows and elimination, are printed. Then, it conducts backward substitution to find the solution vector x, printing the calculations. The final augmented matrix is printed, and the function returns the solution vector.

In [25]:
def gaussian_elimination_pivoting(A, b):
    # Convert A and b to float64 if they are not already
    A = np.array(A, dtype=np.float64)
    b = np.array(b, dtype=np.float64)

    n = len(b)
    
    # Augmenting the matrix A with the vector b
    augmented_matrix = np.hstack((A, b.reshape(-1, 1)))

    print("Original Augmented Matrix:")
    print(augmented_matrix)

    # Forward elimination
    for i in range(n):
        # Partial pivoting
        max_row_index = np.argmax(np.abs(augmented_matrix[i:, i])) + i
        augmented_matrix[[i, max_row_index]] = augmented_matrix[[max_row_index, i]]

        pivot_row = augmented_matrix[i, :]
        print("\nStep", i+1, "- Pivot Row:")
        print(pivot_row)

        for j in range(i + 1, n):
            factor = augmented_matrix[j, i] / pivot_row[i]
            print("  Subtracting", factor, "* Row", i+1, "from Row", j+1)
            augmented_matrix[j, :] -= factor * pivot_row

        print("\nAugmented Matrix after Step", i+1, ":")
        print(augmented_matrix)

    # Backward substitution
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = augmented_matrix[i, -1] / augmented_matrix[i, i]
        print("\nCalculating x[{}]: {} / {} = {}".format(i+1, augmented_matrix[i, -1], augmented_matrix[i, i], x[i]))
        augmented_matrix[:i, -1] -= augmented_matrix[:i, i] * x[i]

    print("\nFinal Augmented Matrix:")
    print(augmented_matrix)

    return x

Example usage for A that is a 6*6 matrix:

A = 

    [[2, -1, 1, 3, 2, -4],

    [1,  3, 2, 1,-1,  2],

    [4,  2,-3,-1, 2,  1],

    [3,  1, 1,-2, 1,  3],

    [1, -4,-1, 2,-1,  1],
    
    [-2, 1, 5,-4, 3,  2]]

b = [9, 1, -2, 6, -4, 7]

In [26]:
A = np.array([[2, -1, 1, 3, 2, -4],
              [1, 3, 2, 1, -1, 2],
              [4, 2, -3, -1, 2, 1],
              [3, 1, 1, -2, 1, 3],
              [1, -4, -1, 2, -1, 1],
              [-2, 1, 5, -4, 3, 2]])

b = np.array([9, 1, -2, 6, -4, 7])

solution = gaussian_elimination_pivoting(A, b)
print("\nSolution:", solution)

Original Augmented Matrix:
[[ 2. -1.  1.  3.  2. -4.  9.]
 [ 1.  3.  2.  1. -1.  2.  1.]
 [ 4.  2. -3. -1.  2.  1. -2.]
 [ 3.  1.  1. -2.  1.  3.  6.]
 [ 1. -4. -1.  2. -1.  1. -4.]
 [-2.  1.  5. -4.  3.  2.  7.]]

Step 1 - Pivot Row:
[ 4.  2. -3. -1.  2.  1. -2.]
  Subtracting 0.25 * Row 1 from Row 2
  Subtracting 0.5 * Row 1 from Row 3
  Subtracting 0.75 * Row 1 from Row 4
  Subtracting 0.25 * Row 1 from Row 5
  Subtracting -0.5 * Row 1 from Row 6

Augmented Matrix after Step 1 :
[[ 4.    2.   -3.   -1.    2.    1.   -2.  ]
 [ 0.    2.5   2.75  1.25 -1.5   1.75  1.5 ]
 [ 0.   -2.    2.5   3.5   1.   -4.5  10.  ]
 [ 0.   -0.5   3.25 -1.25 -0.5   2.25  7.5 ]
 [ 0.   -4.5  -0.25  2.25 -1.5   0.75 -3.5 ]
 [ 0.    2.    3.5  -4.5   4.    2.5   6.  ]]

Step 2 - Pivot Row:
[ 0.   -4.5  -0.25  2.25 -1.5   0.75 -3.5 ]
  Subtracting 0.4444444444444444 * Row 2 from Row 3
  Subtracting 0.1111111111111111 * Row 2 from Row 4
  Subtracting -0.5555555555555556 * Row 2 from Row 5
  Subtracting -0.444

Example usage for A that is a 12*12 matrix:

A =

    [[2, -1, 1, 3, 1, 5, 4, 1, 3, 6, 2, 8],

    [4, 1, -1, 5, 3, 7, 3, 2, 5, 1, 7, 2],

    [6, 3, 2, 8, 5, 9, 6, 4, 1, 3, 5, 1],

    [8, 5, 3, 2, 7, 1, 8, 7, 4, 2, 1, 9],

    [3, 4, 2, 1, 6, 7, 2, 9, 1, 5, 3, 6],

    [1, 2, 6, 4, 8, 3, 1, 5, 9, 7, 2, 4],

    [2, 1, 3, 5, 9, 6, 4, 8, 1, 2, 7, 3],

    [5, 7, 1, 6, 4, 2, 9, 3, 6, 8, 4, 1],

    [9, 8, 4, 7, 2, 3, 7, 1, 8, 4, 9, 5],

    [3, 6, 5, 1, 8, 4, 5, 6, 7, 9, 1, 2],

    [1, 2, 4, 6, 3, 8, 2, 1, 3, 7, 5, 9],
    
    [7, 5, 9, 2, 1, 4, 6, 3, 2, 8, 4, 7]]

b = [7, 15, 29, 24, 21, 15, 8, 12, 20, 18, 23, 16]

In [27]:
A = np.array([[2, -1, 1, 3, 1, 5, 4, 1, 3, 6, 2, 8],
              [4, 1, -1, 5, 3, 7, 3, 2, 5, 1, 7, 2],
              [6, 3, 2, 8, 5, 9, 6, 4, 1, 3, 5, 1],
              [8, 5, 3, 2, 7, 1, 8, 7, 4, 2, 1, 9],
              [3, 4, 2, 1, 6, 7, 2, 9, 1, 5, 3, 6],
              [1, 2, 6, 4, 8, 3, 1, 5, 9, 7, 2, 4],
              [2, 1, 3, 5, 9, 6, 4, 8, 1, 2, 7, 3],
              [5, 7, 1, 6, 4, 2, 9, 3, 6, 8, 4, 1],
              [9, 8, 4, 7, 2, 3, 7, 1, 8, 4, 9, 5],
              [3, 6, 5, 1, 8, 4, 5, 6, 7, 9, 1, 2],
              [1, 2, 4, 6, 3, 8, 2, 1, 3, 7, 5, 9],
              [7, 5, 9, 2, 1, 4, 6, 3, 2, 8, 4, 7]])

b = np.array([7, 15, 29, 24, 21, 15, 8, 12, 20, 18, 23, 16])

solution = gaussian_elimination_pivoting(A, b)
print("\nSolution:", solution)

Original Augmented Matrix:
[[ 2. -1.  1.  3.  1.  5.  4.  1.  3.  6.  2.  8.  7.]
 [ 4.  1. -1.  5.  3.  7.  3.  2.  5.  1.  7.  2. 15.]
 [ 6.  3.  2.  8.  5.  9.  6.  4.  1.  3.  5.  1. 29.]
 [ 8.  5.  3.  2.  7.  1.  8.  7.  4.  2.  1.  9. 24.]
 [ 3.  4.  2.  1.  6.  7.  2.  9.  1.  5.  3.  6. 21.]
 [ 1.  2.  6.  4.  8.  3.  1.  5.  9.  7.  2.  4. 15.]
 [ 2.  1.  3.  5.  9.  6.  4.  8.  1.  2.  7.  3.  8.]
 [ 5.  7.  1.  6.  4.  2.  9.  3.  6.  8.  4.  1. 12.]
 [ 9.  8.  4.  7.  2.  3.  7.  1.  8.  4.  9.  5. 20.]
 [ 3.  6.  5.  1.  8.  4.  5.  6.  7.  9.  1.  2. 18.]
 [ 1.  2.  4.  6.  3.  8.  2.  1.  3.  7.  5.  9. 23.]
 [ 7.  5.  9.  2.  1.  4.  6.  3.  2.  8.  4.  7. 16.]]

Step 1 - Pivot Row:
[ 9.  8.  4.  7.  2.  3.  7.  1.  8.  4.  9.  5. 20.]
  Subtracting 0.4444444444444444 * Row 1 from Row 2
  Subtracting 0.6666666666666666 * Row 1 from Row 3
  Subtracting 0.8888888888888888 * Row 1 from Row 4
  Subtracting 0.3333333333333333 * Row 1 from Row 5
  Subtracting 0.1111111111111